# Faceted search with postgres using tsvectors

In [1]:
__author__ = 'Arbaz Khan'

__TL;DR__ Faceting can add a rich layer of interaction into a database. However, for large databases, it's a challenge to serve faceted-search operations in real-time. A common response is to move to a distributed search engine such as Elasticsearch and parallelize the necessary aggregation operations. However, this might complicate one's infrastructure significantly. In this post, we define and motivate a lightweight alternative: a single-instance postgres set-up with queries that rely on the text-search-vector (`tsvector`) data type to enable faceted-search over medium-sized datasets.

## What is faceted search?

A __facet__ can be defined as a curated division of a dataset, and thus __faceted-search__ is a search operation facilitated via facets to help narrow down search context based on criteria of interest. 

To take a simple example, suppose you are shopping for a house online. A good search engine will allow you to specify filter conditions relating to price, location, number of bedrooms, and so forth. When you set these filter conditions in the search interface, you expect to see a rich breakdown of the matching houses according to the remaining conditions. Faceted-search is what enables this to happen in real time even for very large and complex databases.

It is worth noting that the search aspects of faceted search are just familiar filter operations. The complex part is showing rich useful facets after every search. Thus, our focus here is on building rich dynamic facets that get updated each time the user changes the filter conditions.

The term "faceted-search" gained popularity with the advent and evolution of powerful search frameworks such as [Apache's Lucene](https://lucene.apache.org/). Lucene's documentation also includes [an excellent user's guide](http://lucene.apache.org/core/4_0_0/facet/org/apache/lucene/facet/doc-files/userguide.html) on different aspects of faceted-search.

## Why discuss postgres for faceted-search?

Platforms such as Solr and Elasticsearch make highly performant faceted-search accessible to any application. Both Solr and Elasticsearch use [Lucene's inverted index](https://lucene.apache.org/core/5_1_0/core/org/apache/lucene/codecs/lucene50/package-summary.html#Overview) to make term-based search efficient. Search performance is then further enhanced by distributing the data in a cluster and storing replicas. These are excellent solutions where they are feasible to implement.

However, many web applications rely entirely on relational databases like postgres for data storage and search needs. These solutions are easy to set up in production, even where there are extensive requirements relating to access-control, versioning schema-migrations, and integrating with diverse front-end technologies. 

A major limitation of these solutions, though, is that they are limited when it comes to faceted search. Postgres doesn't straightforwardly support distributed storage or a distributed query engine. Faceted search operations thus tend to be extremely slow, as there are only so many optimizations one can make with a single search index.

In this post, we will show how to overcome some of these limitations by leveraging the functionality of postgresql's [text-search vectors](https://www.postgresql.org/docs/8.3/datatype-textsearch.html) (`tsvectors`). These optimizations enable responsive faceted search for medium-sized datasets – on the order of 100k records with 15-50 metafields, which covers a wide range of common use-cases.

## Problem Definition

Let's continue our house shopping example from above. Suppose each record in the underlying database has certain metadata associated to it, as a set of meta_field–value pairs. For example, `state` is a meta_field and `California`, `Arizona`, etc., are its associated values. The total number of unique meta_field–value pairs is likely to be large and diverse, but we expect the number of distinct meta_fields to be relatively small (say, under 100).

A __filter__ specifies a set of desired meta_field–value pairs. For example, [(`state`, `California`)] is a simple filter that would limit attention to properties in California. This corresponds to a hard constraint that the user might impose via the interface.

A faceted search operation will then consider each of the remaining meta_field–value pairs, returning the number of matching records for them. For example, the meta_field `seller` might take on values like `agent` and `owner`. Faceted search, in the current filter context, would return, for each of these seller values, a count of California properties.

## Traditional SQL solution

In considering an SQL solution, we have to define a data-model representation. Applications are usually constrained by their own schema requirements, and a generic data-model might not work for all use-cases. So, without taking into account any application-specific schema constraints, for a best-case scenario, let's assume an efficient collapsed data model mapping each record to its metadata:

<div id='tab1' style='float:left'>
    <table>
      <tr>
        <th></th>
        <th>Data Type</th>
        <th style="text-align:left">Description</th>          
      </tr>
      <tr>
        <td><b>record_id</b></td>
        <td>UUID</td>
        <td style="text-align:left">Unique identifier to the record</td>          
      </tr>
      <tr>
        <td><b>metadata</b></td>
        <td>jsonb</td>
        <td style="text-align:left">Binary json object mapping each meta_field to its value</td>
      </tr>
    </table>
</div>



Here's what an SQL query for a meta_field `seller` in record's `metadata` would look like, given also the user's filter to California properties:

```
SELECT
  records.metadata -> 'seller' AS meta_field,
  count(*) AS meta_field_count
FROM
  records
WHERE
  records.metadata @> 
   '{"state": "California"}'::jsonb
GROUP BY
  meta_field
ORDER BY
  meta_field_count
LIMIT
  50
```

Note: `@>` is an efficient [jsonb operator](https://www.postgresql.org/docs/9.6/functions-json.html#FUNCTIONS-JSONB-OP-TABLE) that can be indexed to efficiently check for containment of one jsonb record with respect to another.

You would repeat the above query for all remaining meta_fields to meet the requirements of faceted-search.

While this is a working solution, performance for the above query is very poor. As we show later, it can take nearly  10 seconds to get a response for a dataset of 100k records with 10+ meta_fields. Performance improvements are needed.

## Redefining the problem using tsvectors

When full text-search was introduced into postgresql, it brought two new datatypes: [tsvector](https://www.postgresql.org/docs/9.6/datatype-textsearch.html#DATATYPE-TSVECTOR) for storing preprocessed documents and [tsquery](https://www.postgresql.org/docs/9.6/datatype-textsearch.html#DATATYPE-TSQUERY) for textual queries. 

`tsvector` is defined as a sorted list of unique lexemes and makes text-search work faster than the `LIKE` operator, while `tsquery` is an expression on lexemes with boolean operators between them. You can convert text and arrays to `tsvectors` and perform other specific operations on them – most prominently, text search. 

With `tsvector`, `tsquery`, and the full-text-search (FTS) operator `@@`, you can determine if certain combination of words are found in a text document. Here's a simple example:
```
=# select 'charming 3 bedroom with pool'::tsvector @@ 'charming & pool'::tsquery;
?column?
----------
t
```
This tests if  `charming` and `pool` both appear (`&` operator) in the sentence `charming 3 bedroom with pool`. When a text is cast to `tsvector`, lexemes are extracted out to build a sparse representation of the sentence on which text queries can be run efficiently.

Faceted-search amounts to looking at the frequency of occurence of meta_field–value pairs. Thus, if we model each such pair as a word instead, this becomes a word-count problem. Building on this, a record becomes a set of words and thus can be represented as a text document. Groups of records combined together are still text documents. Filters in turn become search queries (`ts_query`) on `tsvector` objects. We can rely on [ts_stat](https://www.postgresql.org/docs/9.6/functions-textsearch.html) to count the words themselves.

To implement this approach, we need a model that looks like this:

<div id='tab2' style='float:left'>
    <table>
      <tr>
        <th></th>
        <th>Data Type</th>
        <th style="text-align:left">Description</th>
      </tr>
      <tr>
        <td><b>record_id</b></td>
        <td>UUID</td>
        <td style="text-align:left">Unique identifier to the record</td>
      </tr>
      <tr>
        <td><b>record_tsv</b></td>
        <td>tsvector</td>
        <td style="text-align:left">text-search vector representation of meta_field–value pairs encoded as words</td>          
      </tr>
    </table>
</div>

We encode each meta_field–value pair in a record as a word and combine words together to form a document. Finally, we turn this document to a text-search vector. (Aside: Even if you are bound by application schema constraints, you can build a [materialized view](https://www.postgresql.org/docs/9.6/rules-materializedviews.html) from your tables so that the data representation minimally has the above columns.)

Here is a representative SQL-query for all meta_fields combined, again given the user constraint to look only at California properties:

```
select meta_field, value, ndoc as freq
/* Parse result of word-counts to get top-50 values
for each `meta_field` by frequency of occurence */    
from                                              
  (                                       
    select
      -- Break encoded words to get `meta_field`
      split_part(word, ':', 1) as meta_field,    
      split_part(word, ':', 2) as value,
      ndoc,
      -- Rank each `value` of a `meta_field` by frequency of occurence
      row_number() OVER (                         
        partition by split_part(word, ':', 1)
        order by ndoc desc
      ) as rank
    from
      -- Word-count on all qualified records       
      ts_stat(                                   
        $$
        select
          record_tsv
        from
          records
          -- Filter records based on the given criteria
          where record_tsv @@ to_tsquery('state:California')
   $$
      )
  ) meta_field_value_counts
where
  rank <= 50;
```      
Here, preprocessing is required to convert each record's metadata to a `tsvector`. For instance, the `to_tsquery` function forms a word `state:California` from the meta_field–value pair `("state", "California")`. 

If you have special characters in your meta_field and/or value, it's recommended to build `tsvectors` by first building an array of words and then using [array_to_tsvector](https://www.postgresql.org/docs/current/functions-textsearch.html#TEXTSEARCH-FUNCTIONS-TABLE) to construct the `tsvector` to ensure that lexemes extracted from the word are identical to the word itself (and not some reduced form of the word).

The query looks complicated, but the performance gains we get out of this are significant.

## Evaluation

We now report on a series of experiments comparing the Traditional SQL approach, our `tsvectors` approach, and Elasticsearch.

It should be noted that query performance for all these approaches depends on the coverage of the selected facet. That is, query performance for a facet that covers just 0.1% of the dataset will be vastly better than a facet that covers 50% of the same dataset. 

### Comparison with the Traditional SQL approach

We use a database of 100K records with 15 meta_fields in each record. To optimize performance for the Traditional SQL approach, we used a [GIN index](https://www.postgresql.org/docs/9.4/datatype-json.html#JSON-INDEXING) on the `metadata` column with `jsonb_path_ops` as the operator class. For the `tsvectors` approach, we used a [GIN index](https://www.postgresql.org/docs/9.6/textsearch-indexes.html) for text-search on the `record_tsv` column.

The following figure shows that, as the facet's coverage increases, `tsvectors` shows increasingly large gains over the Traditional SQL approach:

<img src="img/traditional-vs-tsvectors100k.png" alt="drawing" width="700" />

We use CPU time and not wall-time to measure performance, as one can set up concurrent SQL threads (one for each meta_field query) to make the operation return results quickly. However, the database's resources are unavailable for other queries during that time.

### Comparison with Elasticsearch

Given that `tsvectors` shows significant performance gains over the Traditional SQL approach, the question remains whether it can replace a distributed Lucene-powered search engine such as Elasticsearch.

We compare a single-instance postgres database  with a three-node Elasticsearch cluster. And, for these experiments, we increase the database size to 1 million records and about 80 field-value pairs per record on average. Postgres database was deployed on a db4.large RDS instance while each node of the Elasticsearch cluster was deployed on a r3.xlarge EC2 instance.

Here are the results:

<img src="img/tsvectors-vs-Elasticsearch-1M.png" alt="drawing" width="700" />

We have used wall times (and not CPU-time) to measure performance. This is because ES-clusters aren't resource-bound and can be scaled according to needs, and thus wall-time can be optimized. 

From the results above, we can see that response times for the postgres query can reach 70 seconds. Usually, web applications have a response timeout when a query takes more than a minute. This  renders postgres unusable for faceted-search on datasets of size 1M or more. Elasticsearch, in contrast, can be scaled by creating as many nodes as one could want (in theory) and can thus overcome performance limitations with huge datasets.

## Conclusion

We defined three methods for implementing faceted search: using a distributed search engine like Elasticsearch, using traditional SQL methods, and relying on `tsvectors` to reduce faceted search to a text-search problem within postgres. Traditional SQL methods are too slow to support full faceted search. While the Elasticsearch approach is clearly the superior choice, especially for very large databases, we showed that our `tsvectors`-based method is viable for medium-sized datasets.

Thus, if your database is medium-sized, you needn't complicate your database infrastructure to support faceted search. However, if you anticipate the scale of datasets employed for faceted-search to extend in the order of millions, you will have to prepare yourself for the move to Elasticsearch or a similar distributed-search framework. Tools such as [logstash](https://www.elastic.co/blog/logstash-jdbc-input-plugin) make it easy to migrate data, but application-code and the data-platform set-up become the greatest liability when making such a move. Using a logical data layer between application and data-storage can keep application-code-related setbacks to minimum and help facilitate a smooth transition.